The goal is to make sure code runs as intended and measure performance improvements in relation to old code


PART 1: TESTING

In [1]:
import os
import sys

cwd = os.getcwd()

cwd = os.path.abspath(os.path.join(cwd, "..", "..")) + "/src"

print(cwd)
sys.path.append(cwd)

/home/mikael/GETELEC/src


In [5]:
from getelec import GETELECModel as GTM

In [6]:
gtm = GTM()

Now let's see what properties the model has before we do anything

In [7]:
gtm.getParameters()

{'bandGap': None,
 'conductionBandBottom': None,
 'effectiveMassConduction': None,
 'effectiveMassValence': None,
 'emitter': None,
 'emitterType': None,
 'field': None,
 'gamma': None,
 'numberOfSpectrumPoints': None,
 'radius': None,
 'temperature': None,
 'workFunction': None}

Let's create some dummy data

In [8]:
import numpy as np

field = np.array([1, 2, 3, 4, 5])
radius = np.full(5, 50)
gamma = np.full(5, 10)
workFunction = np.full(5, 4)
temperature = np.full(5, 300)

conductionBandBottom = 4.05
fermiLevel = 4.61
bandGap = 1.12
effectiveMassConduction = 0.7
effectiveMassValence = 0.5

numberOfSpectrumPoints = 8


TEST 1: calling properties that were not yet calculated

In [9]:
print(gtm.getCurrentDensity())
print(gtm.getElectronSpectrum())
print(gtm.getNottinghamHeat())

WARNING, you have asked currentDensity without calculating it first. You might want to run getelecModel.run(calculateCurrentDensity=True) first
None
WARNING, you have asked Electron Spectrum without calculating it first. You might want to run getelecModel.run(calculateElectronSpectrum=True) first
None
WARNING, you have asked Nottingham heat without calculating it first. You might want to run getelecModel.run(calculateNottinghamHeat=True) first
None


In [10]:
gtm.getParameters()

{'bandGap': None,
 'conductionBandBottom': None,
 'effectiveMassConduction': None,
 'effectiveMassValence': None,
 'emitter': None,
 'emitterType': None,
 'field': None,
 'gamma': None,
 'numberOfSpectrumPoints': None,
 'radius': None,
 'temperature': None,
 'workFunction': None}

TEST 2: assigning property in model instantiation (METALS)

In [11]:
gtm = GTM(emitterType = 'metal', field = field, radius = radius, gamma = gamma, workFunction = workFunction, temperature = temperature, numberOfSpectrumPoints = numberOfSpectrumPoints)

In [14]:
gtm.getParameters()

{'bandGap': None,
 'conductionBandBottom': None,
 'effectiveMassConduction': None,
 'effectiveMassValence': None,
 'emitter': <getelec.MetalEmitter at 0x7f6b46c0b850>,
 'emitterType': 'metal',
 'field': array([1, 2, 3, 4, 5]),
 'gamma': array([10, 10, 10, 10, 10]),
 'numberOfSpectrumPoints': 8,
 'radius': array([50, 50, 50, 50, 50]),
 'temperature': array([300, 300, 300, 300, 300]),
 'workFunction': array([4, 4, 4, 4, 4])}

TEST 3: running all calculations

In [15]:
gtm.run(calculateCurrent=True, calculateNottinghamHeat=True, calculateSpectrum=True)

print(f'Current Density: ${gtm.getCurrentDensity()},\nNottingham Heat: ${gtm.getNottinghamHeat()},\nElectronSpectrum: ${gtm.getElectronSpectrum()}')

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mikael/GETELEC/src/getelec.py", line 1504, in __run_metal_worker
    results[idx] = {'currentDensity': currentDensity, 'nottinghamHeat': nottinghamHeat, 'electronEnergy': energy, 'electronCount': electronCount}
IndexError: list assignment index out of range


TypeError: 'NoneType' object is not subscriptable

In [11]:
gtm.getParameters()

{'bandGap': None,
 'conductionBandBottom': None,
 'effectiveMassConduction': None,
 'effectiveMassValence': None,
 'emitter': <getelec.MetalEmitter at 0x7f2bc54cbc70>,
 'emitterType': 'metal',
 'fermiLevel': None,
 'field': array([1, 2, 3, 4, 5]),
 'gamma': array([10, 10, 10, 10, 10]),
 'numberOfSpectrumPoints': 8,
 'radius': array([50, 50, 50, 50, 50]),
 'temperature': array([300, 300, 300, 300, 300]),
 'workFunction': array([4, 4, 4, 4, 4])}

TEST 4: see what happens when data changes

In [12]:
print(gtm.field)
gtm.setParameters(field = np.array([2, 3, 4, 5, 6]))
print(gtm.field)

[1 2 3 4 5]
[2 3 4 5 6]


In [13]:
gtm.getCurrentDensity()

WARNING, you have asked currentDensity without calculating it first. You might want to run getelecModel.run(calculateCurrentDensity=True) first


In [14]:
gtm.calculateCurrentDensity()
print(gtm.getCurrentDensity())

[4.99385567e-16 1.06113266e-11 1.57722206e-09 3.26236555e-08
 2.50997005e-07]


TEST 5: instantiating SEMICONDUCTOR

In [15]:
gtm = GTM()

gtm.setParameters(emitterType='semiconductor', field = field, radius = radius, gamma = gamma, conductionBandBottom = conductionBandBottom, fermiLevel = fermiLevel, bandGap = bandGap, effectiveMassConduction = effectiveMassConduction, effectiveMassValence = effectiveMassValence, numberOfSpectrumPoints = 8, temperature = temperature, workFunction = workFunction)

In [16]:
gtm.getParameters()
print(gtm.emitter)

In [17]:
gtm.run(True, True, True)

print(f'Current Density: ${gtm.getCurrentDensity()},\nNottingham Heat: ${gtm.getNottinghamHeat()},\nElectronSpectrum: ${gtm.getElectronSpectrum()}')

Current Density: $[-1.20435449e-25  6.09906305e-19  9.99617995e-14  3.98299313e-11
  1.47354071e-09],
Nottingham Heat: $[-1.10671221e-25  3.80027245e-19  6.84328117e-14  2.96438993e-11
  1.18186792e-09],
ElectronSpectrum: ${'energy': [array([-2.36865954, -2.05721629, -1.74577303, -1.43432977, -1.12288651,
       -0.81144326, -0.5       ,  2.97790311,  3.02580622,  3.07370933,
        3.12161245,  3.16951556,  3.21741867,  3.26532178]), array([-2.36865954, -2.05721629, -1.74577303, -1.43432977, -1.12288651,
       -0.81144326, -0.5       ,  2.96674774,  3.00349547,  3.04024321,
        3.07699094,  3.11373868,  3.15048642,  3.18723415]), array([-2.36865954, -2.05721629, -1.74577303, -1.43432977, -1.12288651,
       -0.81144326, -0.5       ,  2.96414346,  2.99828691,  3.03243037,
        3.06657382,  3.10071728,  3.13486074,  3.16900419]), array([-2.36865954, -2.05721629, -1.74577303, -1.43432977, -1.12288651,
       -0.81144326, -0.5       ,  2.96297262,  2.99594525,  3.02891787,
      

In [18]:
new_gtm = GTM('metal', field=field, radius=radius, gamma=gamma, workFunction=workFunction, temperature=temperature)

In [19]:
new_gtm.calculateCurrentDensity().currentDensity

array([2.15675678e-27, 4.99385567e-16, 1.06113266e-11, 1.57722206e-09,
       3.26236555e-08])